In [1]:
%cd ~/corpus

/home/sibava/corpus


In [2]:
import json
from transformers import T5Tokenizer
import pprint
import copy
import itertools
from tqdm.auto import tqdm
import glob
import os

/home/sibava/miniconda3/envs/psat5/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = T5Tokenizer.from_pretrained('megagonlabs/t5-base-japanese-web')

In [9]:
tokenizer.decode([18336])

'発見した'

In [4]:
case_en_ja = {
	'ga':'が',
	'o':'を',
	'ni':'に',
	'yotte':'によって'
}

In [6]:
input_path_list = glob.glob("/home/sibava/corpus/ntc-train-with-juman-A/*")
output_dir_path = "/home/sibava/PAS-T5/datasets/pas-dataset-closest"

for input_path in input_path_list:	
	lines =  open(input_path,mode='r').readlines()
	outputfile_path = os.path.join(output_dir_path,os.path.basename(input_path))
	outputfile = open(outputfile_path,mode='w')
	for line in tqdm(lines):
		doc = json.loads(line)
		sents = doc["sents"]
		pas_list = doc["pas_list"]
		for pas in pas_list:
			pas_dict = json.loads(pas)
			predicate = "".join([sents[pas_dict["prd_sent_idx"]][i] for i in pas_dict["predicate_positions"]])
			predicate_positions = pas_dict["predicate_positions"]
			prd_sent_idx = pas_dict["prd_sent_idx"]
			alt_type = pas_dict["alt_type"]
			context = copy.deepcopy(sents[0:pas_dict['prd_sent_idx'] + 1])
			context[prd_sent_idx].insert(predicate_positions[0],"<extra_id_0>")
			context[prd_sent_idx].insert(predicate_positions[-1] + 2,"<extra_id_1>")
			argument_and_case = ''
			gold_arguments = {}
			case_types = {}
			for gold in pas_dict["gold_labels"]:
				case_type = gold['case_type']
				case_name = gold['case_name']
				case_types[case_name] = case_type
				# if case_type == 'null':
				if (case_type in ['null']):
					gold_arguments[case_name] = []
					continue
				elif case_type.startswith('exog'):
					golds_chain = ['<extra_id_99>']
					closest_gold_argument = '<extra_id_99>'
				elif case_type.startswith('exo1'):
					golds_chain = ['<extra_id_98>']
					closest_gold_argument = '<extra_id_98>'
				elif case_type.startswith('exo2'):
					golds_chain = ['<extra_id_97>']
					closest_gold_argument = '<extra_id_97>'
				else:
					golds_chain = [sents[x['sent_idx']][x['word_idx']] for x in gold['gold_positions']]
					for x in gold['gold_positions']:
						if(x['case_type'] == case_type):
							closest_gold_argument = sents[x['sent_idx']][x['word_idx']] 
							#the gold argument that is closest to predicate.In inter setting,the first argument is selected.
							break
				argument_and_case += closest_gold_argument + case_en_ja[case_name]
				gold_arguments[case_name] = list(set(golds_chain))
			concated_context = ''.join(list(itertools.chain.from_iterable(context))) + tokenizer.eos_token
			padded_input_ids = tokenizer(concated_context,max_length=512,padding="max_length").input_ids
			t5_pas_dict = {'input_ids':padded_input_ids[len(padded_input_ids)-512:]}
			t5_pas_dict['input_tokens'] = concated_context
			t5_pas_dict['labels'] = tokenizer(argument_and_case + predicate,max_length=30,padding="max_length").input_ids
			t5_pas_dict['gold_arguments'] = gold_arguments
			t5_pas_dict['case_types'] = case_types
			t5_pas_dict['predicate'] = predicate
			t5_pas_dict['alt_type'] = alt_type
			outputfile.write(json.dumps(t5_pas_dict) + '\n')
	outputfile.close()


  0%|          | 0/1751 [00:00<?, ?it/s]/home/sibava/miniconda3/envs/psat5/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:194: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
100%|██████████| 696/696 [00:28<00:00, 24.15it/s]


In [7]:
pprint.pprint(json.loads(pas_list[4]))

{'alt_type': 'active',
 'gold_labels': [{'case_name': 'ga',
                  'case_type': 'dep',
                  'gold_positions': [{'case_type': 'dep',
                                      'sent_idx': 2,
                                      'word_idx': 5}]},
                 {'case_name': 'o', 'case_type': 'null', 'gold_positions': []},
                 {'case_name': 'ni',
                  'case_type': 'null',
                  'gold_positions': []}],
 'prd_sent_idx': 2,
 'predicate_positions': [2, 3]}
